In [1]:
import os
from openai import AzureOpenAI

azure_endpoint = "https://5team-history-azureai-services.openai.azure.com/"
api_version = "2024-12-01-preview"
deployment_name = "gpt-35-turbo"
subscription_key = "apikey"

# 클라이언트 생성
client = AzureOpenAI(
    api_key=subscription_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

system_message = (
    "너는 조선시대 한국사 전문가 AI야. 항상 RAG나 조선왕조실록 등 검증된 사료에 기반해서만 답변해야 하며, "
    "추측이 포함될 경우 반드시 '추측입니다'라고 명시해. 출처가 명확하지 않거나 불분명한 경우는 '알 수 없습니다'라고 답해야 해. "
    "사용자의 질문이 모호하다면, 추가 정보를 정중하게 요청해. 반드시 사실과 고증에 근거해 정확하고 책임감 있는 답변을 제공해줘."
)

def extract_keywords(query):
    prompt = f"다음 문장에서 핵심 키워드 3~5개만 '키워드1, 키워드2, ...' 형식으로 추출해줘:\n{query}\n>"
    response = client.chat.completions.create(
        model=deployment_name,
        max_tokens=100,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    keywords = response.choices[0].message.content.strip()
    return [kw.strip() for kw in keywords.split(",")]

def evaluate_answer(keywords, answer):
    eval_prompt = (
        f"다음 키워드: {', '.join(keywords)}\n아래 답변이 키워드를 잘 반영하고 있는지 "
        f"조선시대 한국사 전문가 AI 시각으로 평가해줘:\n{answer}"
    )
    response = client.chat.completions.create(
        model=deployment_name,
        max_tokens=200,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": eval_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

def get_answer(query):
    rag_prompt = (
        f"{query}\n(다음 내용은 조선왕조실록을 참조하여 고증한 내용으로 작성되어야 합니다.)"
    )
    response = client.chat.completions.create(
        model=deployment_name,
        max_tokens=1024,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": rag_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    query = "조선 후기의 양반"

    # 1. 키워드 추출
    keywords = extract_keywords(query)

    # 2. 질문에 대한 답변 생성
    answer = get_answer(query)

    # 3. AI가 답변과 키워드 매칭 평가
    ai_eval = evaluate_answer(keywords, answer)

    # 4. JSON 결과 구성 및 출력
    result_json = {
        "keywords": keywords,
        "ai_eval": ai_eval,
        "answer": answer,
    }
    print(result_json)


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}